# Base

In [ ]:
%%capture
!apt-get install maven -qq

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import json
import glob
import sys

In [ ]:
%%capture
!git clone --recurse-submodules https://github.com/castorini/anserini.git
%cd anserini
!cd tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!mvn clean package appassembler:assemble -DskipTests -Dmaven.javadoc.skip=true

In [ ]:
#import the necessary data (i am not sure this is the latest data)
!mkdir collections/fever
!mkdir indexes/fever

!wget https://s3-eu-west-1.amazonaws.com/fever.public/feverous/feverous-wiki-pages.zip -P collections/fever
!unzip collections/fever/feverous-wiki-pages.zip -d collections/fever

!wget https://s3-eu-west-1.amazonaws.com/fever.public/feverous/train.jsonl -P collections/fever
!wget https://s3-eu-west-1.amazonaws.com/fever.public/feverous/dev.jsonl -P collections/fever

--2021-07-19 19:27:49--  https://s3-eu-west-1.amazonaws.com/fever.public/feverous/feverous-wiki-pages.zip
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.92.163
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.92.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9906569155 (9.2G) [application/zip]
Saving to: ‘collections/fever/feverous-wiki-pages.zip’

feverous-wiki-pages 100%[===================>]   9.23G  21.4MB/s    in 7m 20s  

2021-07-19 19:35:10 (21.5 MB/s) - ‘collections/fever/feverous-wiki-pages.zip’ saved [9906569155/9906569155]

Archive:  collections/fever/feverous-wiki-pages.zip
   creating: collections/fever/FeverousWikiv1/
  inflating: collections/fever/FeverousWikiv1/wiki_283.jsonl  
  inflating: collections/fever/FeverousWikiv1/wiki_402.jsonl  
  inflating: collections/fever/FeverousWikiv1/wiki_177.jsonl  
  inflating: collections/fever/FeverousWikiv1/wiki_287.jsonl  
  inflating: collect

In [ ]:
!ls collections/fever/FeverousWikiv1

wiki_000.jsonl	wiki_109.jsonl	wiki_218.jsonl	wiki_327.jsonl	wiki_436.jsonl
wiki_001.jsonl	wiki_110.jsonl	wiki_219.jsonl	wiki_328.jsonl	wiki_437.jsonl
wiki_002.jsonl	wiki_111.jsonl	wiki_220.jsonl	wiki_329.jsonl	wiki_438.jsonl
wiki_003.jsonl	wiki_112.jsonl	wiki_221.jsonl	wiki_330.jsonl	wiki_439.jsonl
wiki_004.jsonl	wiki_113.jsonl	wiki_222.jsonl	wiki_331.jsonl	wiki_440.jsonl
wiki_005.jsonl	wiki_114.jsonl	wiki_223.jsonl	wiki_332.jsonl	wiki_441.jsonl
wiki_006.jsonl	wiki_115.jsonl	wiki_224.jsonl	wiki_333.jsonl	wiki_442.jsonl
wiki_007.jsonl	wiki_116.jsonl	wiki_225.jsonl	wiki_334.jsonl	wiki_443.jsonl
wiki_008.jsonl	wiki_117.jsonl	wiki_226.jsonl	wiki_335.jsonl	wiki_444.jsonl
wiki_009.jsonl	wiki_118.jsonl	wiki_227.jsonl	wiki_336.jsonl	wiki_445.jsonl
wiki_010.jsonl	wiki_119.jsonl	wiki_228.jsonl	wiki_337.jsonl	wiki_446.jsonl
wiki_011.jsonl	wiki_120.jsonl	wiki_229.jsonl	wiki_338.jsonl	wiki_447.jsonl
wiki_012.jsonl	wiki_121.jsonl	wiki_230.jsonl	wiki_339.jsonl	wiki_448.jsonl
wiki_013.jsonl	wiki_122.j

In [ ]:
# A folder path, a file path, or a list of file paths is needed

# DATA_LOCATION = "wiki_pages"
# DATA_LOCATION = "wiki_pages/wiki_000.jsonl"
DATA_LOCATION = [
    "collections/fever/FeverousWikiv1/wiki_000.jsonl",
    "collections/fever/FeverousWikiv1/wiki_001.jsonl",
    "collections/fever/FeverousWikiv1/wiki_002.jsonl",
    "collections/fever/FeverousWikiv1/wiki_003.jsonl",
    "collections/fever/FeverousWikiv1/wiki_004.jsonl",
]
OUTPUT_FOLDER = "wiki_pages_anserini"

SIMPLIFY_PAGE_LINKS = False
PREFER_URL_FOR_LINKS = False

# If SIMPLIFY_PAGE_LINKS and not PREFER_URL_FOR_LINKS: [[Page_(disambiguation)|Page Title]] -> Page Title
# If SIMPLIFY_PAGE_LINKS and PREFER_URL_FOR_LINKS: [[Page_(disambiguation)|Page Title]] -> Page (disambiguation)
# Note that the URL basename can sometimes contain extra information, but it can also redirect to a page that is different than the visible label.

SKIP_TABLES = True
SKIP_SECTIONS = True
SKIP_LISTS = True

MERGE_CHILDREN = False

SUBSECTION_SEP = " ~~"
SUBSUBSECTION_SEP = " ~~"
# ID_SEP = "\t"
ID_SEP = " "
# ITEM_SEP = "\n"
ITEM_SEP = " ~~"

import glob
import os

to_process = None

if type(DATA_LOCATION) == list:  # A list of file paths
    to_process = [file for file in DATA_LOCATION]
elif os.path.isfile(DATA_LOCATION):  # A file path
    to_process = [DATA_LOCATION]
elif os.path.isdir(DATA_LOCATION):  # A folder
    to_process = glob.glob("{}/*.jsonl".format(DATA_LOCATION))
else:
    raise ValueError("Data location is not a valid file or folder.")

print("{} file(s) will be processed.".format(len(to_process)))

import json
import re
from timeit import default_timer as timer


def clean_value(value):
    """Replaces unnecessary space (including \t and such) with a single space
    character.

    Arg:
        value: A string.

    Returns:
        The simplified version of the provided string.
    """
    return " ".join(value.split())


def final_clean(
    field,
    simplify_page_links=SIMPLIFY_PAGE_LINKS,
    prefer_url_for_links=PREFER_URL_FOR_LINKS,
):
    """Removes multiple space characters with a single space character. Removes
    leading and trailing spaces. Simplifies the page links according to the
    parameters.

    Args:
        field (str): A field of the processed page data.
        simplify_page_links (Boolean): A Boolean indicating whether the page
            links must be simplified. By default, it is set to
            SIMPLIFY_PAGE_LINKS.
        prefer_url_for_links (Boolean): A Boolean indicating whether the page
            links must be simplified to the URL basename instead of the link's
            label. By default, it is set to SIMPLIFY_PAGE_LINKS.

    Returns:
        str: The simplified version of the provided string.
    """
    field = re.sub(" +", " ", field.strip(), flags=re.MULTILINE)
    if SIMPLIFY_PAGE_LINKS and PREFER_URL_FOR_LINKS:
        # Simplifies links by replacing them with the URL basename (replaces underscores)
        # Underscores are replaced with space
        field = re.sub(r"(?:\[\[)(.*)?(?:\|)", field.replace("_", " "), field)
        field = re.sub(r"(\|).*?(\]\])|\[\[", "", field, flags=re.MULTILINE)
    elif SIMPLIFY_PAGE_LINKS:
        # Simplifies links by replacing them with the titles
        field = re.sub(r"(\[\[).*?(\|)|]]", "", field, flags=re.MULTILINE)
    return field


if not os.path.exists(OUTPUT_FOLDER):  # Creates the output folder if it does not exist
    os.mkdir(OUTPUT_FOLDER)

timer_start = timer()

counter = 0
for file_path in to_process:
    file_basename = os.path.basename(file_path)
    output_path = os.path.join(OUTPUT_FOLDER, file_basename)

    open(output_path, mode="w", encoding="utf-8").close()  # Creating an empty output file

    with open(file_path, "r", encoding="utf-8") as f:
        file_data = [json.loads(line) for line in f]

    for page in file_data:

        page_data_processed = {"id": page["title"], "text": "", "lines": ""}

        # The order list of the page will be used to parse the items
        for item_id, item in enumerate(page["order"]):
            if item.startswith("sentence_"):  # Sentence
                clean_element = clean_value(page[item])
                page_data_processed["text"] += " {}".format(clean_element)
                page_data_processed["lines"] += " {}{}{}{}".format(item, ID_SEP, clean_element, ITEM_SEP)
            elif item.startswith("table_"):  # Table
                if SKIP_TABLES:
                    continue
                page_data_processed["text"] += " "
                page_data_processed["lines"] += "{}{}".format(item, ID_SEP)
                table_rows = []
                for row in page[item]["table"]:
                    row_items = []
                    for cell in row:
                        if MERGE_CHILDREN:
                            row_items.append(clean_value(cell["value"]))
                        else:
                            page_data_processed["text"] += " {}".format(clean_value(cell["value"]))
                            page_data_processed["lines"] += " {}{}{}{}".format(cell["id"], ID_SEP, clean_value(cell["value"]), ITEM_SEP)

                    if MERGE_CHILDREN:
                        row_text = "{}".format(SUBSUBSECTION_SEP).join(row_items)
                        table_rows.append(row_text)

                if "caption" in page[item]:
                    if MERGE_CHILDREN:
                        table_rows.append(clean_value(page[item]["caption"]))
                    else:
                        page_data_processed["text"] += " {}".format(clean_value(page[item]["caption"]))
                        page_data_processed["lines"] += " {}{}{}{}".format(item_id, ID_SEP, clean_value(page[item]["caption"]), ITEM_SEP)

                if MERGE_CHILDREN:
                    table_text = "{}".format(SUBSECTION_SEP).join(table_rows)
                    page_data_processed["text"] += table_text
                    page_data_processed["lines"] += table_text
                    page_data_processed["lines"] += ITEM_SEP
            elif item.startswith("section_"):  # Section
                if SKIP_SECTIONS:
                    continue
                clean_element = clean_value(page[item]["value"])
                page_data_processed["text"] += " {}".format(clean_element)
                page_data_processed["lines"] += " {}{}{}{}".format(item, ID_SEP, clean_element, ITEM_SEP)
            elif item.startswith("list_"):  # List
                if SKIP_LISTS:
                    continue
                list_items = []
                for list_item in page[item]["list"]:
                    if MERGE_CHILDREN:
                        list_items.append(clean_value(list_item["value"]))
                    else:
                        page_data_processed["text"] += " {}".format(clean_value(list_item["value"]))
                        page_data_processed["lines"] += " {}{}{}{}".format(list_item["id"], ID_SEP, clean_value(list_item["value"]), ITEM_SEP)
                
                if MERGE_CHILDREN:
                    list_text = "{}".format(SUBSECTION_SEP).join(list_items)
                    page_data_processed["text"] += list_text
                    page_data_processed["lines"] += " {}{}{}{}".format(item, ID_SEP, list_text, ITEM_SEP)
            else:  # All alternatives must be handled and the code must not reach here
                raise ValueError("Unidentified page element found.")

        page_data_processed["text"] = final_clean(page_data_processed["text"])
        page_data_processed["lines"] = final_clean(page_data_processed["lines"])

        # Appending the processed page data to the file
        with open(output_path, "a", encoding="utf-8") as f:
            json.dump(page_data_processed, f, ensure_ascii=False)
            f.write("\n")

timer_end = timer()

print("Formatting is complete after {:.2f} minute(s)".format((timer_end - timer_start)/60))

5 file(s) will be processed.
Formatting is complete after 0.67 minute(s)


In [ ]:
!sh target/appassembler/bin/IndexCollection -collection FeverParagraphCollection \
 -input wiki_pages_anserini \
 -index indexes/fever/lucene-index-fever-paragraph \
 -generator DefaultLuceneDocumentGenerator \
 -threads 1 -storePositions -storeDocvectors -storeRaw \
  >& logs/log.fever &

In [ ]:
!tar -cvf "/content/anserini/indexes/fever/lucene-index-fever-paragraph.tar" "/content/anserini/indexes/fever/lucene-index-fever-paragraph" #Tar operation for exporting the index and make it reusable
# Only one indexing will be enough we will use the tar file after that use the code below to untar it and equate to the LucuneDir
#!tar xvfz lucene-index-fever-paragraph.tar

tar: Removing leading `/' from member names
/content/anserini/indexes/fever/lucene-index-fever-paragraph/
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1.nvm
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1.si
/content/anserini/indexes/fever/lucene-index-fever-paragraph/segments_2
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1.fnm
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1_Lucene50_0.doc
/content/anserini/indexes/fever/lucene-index-fever-paragraph/write.lock
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1_Lucene80_0.dvm
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1.fdt
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1_Lucene50_0.tip
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1.tvd
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1.nvd
/content/anserini/indexes/fever/lucene-index-fever-paragraph/_1.tvx
/content/anserini/indexes/fever/lucene

In [ ]:
luceneDir = 'indexes/fever/lucene-index-fever-paragraph'

In [ ]:
USE_SUMMARY = False

In [ ]:
import os
#%%capture
!curl -O https://download.java.net/java/GA/jdk11/9/GPL/openjdk-11.0.2_linux-x64_bin.tar.gz
!mv openjdk-11.0.2_linux-x64_bin.tar.gz /usr/lib/jvm/; cd /usr/lib/jvm/; tar -zxvf openjdk-11.0.2_linux-x64_bin.tar.gz
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/jdk-11.0.2/bin/java 1
!update-alternatives --set java /usr/lib/jvm/jdk-11.0.2/bin/java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/jdk-11.0.2"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  178M  100  178M    0     0   141M      0  0:00:01  0:00:01 --:--:--  141M
jdk-11.0.2/bin/jaotc
jdk-11.0.2/bin/jar
jdk-11.0.2/bin/jarsigner
jdk-11.0.2/bin/java
jdk-11.0.2/bin/javac
jdk-11.0.2/bin/javadoc
jdk-11.0.2/bin/javap
jdk-11.0.2/bin/jcmd
jdk-11.0.2/bin/jconsole
jdk-11.0.2/bin/jdb
jdk-11.0.2/bin/jdeprscan
jdk-11.0.2/bin/jdeps
jdk-11.0.2/bin/jhsdb
jdk-11.0.2/bin/jimage
jdk-11.0.2/bin/jinfo
jdk-11.0.2/bin/jjs
jdk-11.0.2/bin/jlink
jdk-11.0.2/bin/jmap
jdk-11.0.2/bin/jmod
jdk-11.0.2/bin/jps
jdk-11.0.2/bin/jrunscript
jdk-11.0.2/bin/jshell
jdk-11.0.2/bin/jstack
jdk-11.0.2/bin/jstat
jdk-11.0.2/bin/jstatd
jdk-11.0.2/bin/keytool
jdk-11.0.2/bin/pack200
jdk-11.0.2/bin/rmic
jdk-11.0.2/bin/rmid
jdk-11.0.2/bin/rmiregistry
jdk-11.0.2/bin/serialver
jdk-11.0.2/bin/unpack200
jdk-11.0.2/conf/logging.properties
jdk-11.0.2/conf/management/jmx

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
!mkdir ~/kaggle
!mkdir ~/kaggle/working/
!mkdir ~/kaggle/working/sentence_wise_email/
!mkdir ~/kaggle/working/sentence_wise_email/module/
!mkdir ~/kaggle/working/sentence_wise_email/module/module_useT
# Download the module, and uncompress it to the destination folder. 
!curl -L "https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed" | tar -zxvC ~/kaggle/working/sentence_wise_email/module/module_useT

mkdir: cannot create directory ‘/root/kaggle’: File exists
mkdir: cannot create directory ‘/root/kaggle/working/’: File exists
mkdir: cannot create directory ‘/root/kaggle/working/sentence_wise_email/’: File exists
mkdir: cannot create directory ‘/root/kaggle/working/sentence_wise_email/module/’: File exists
mkdir: cannot create directory ‘/root/kaggle/working/sentence_wise_email/module/module_useT’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
./
./tfhub_module.pb
./variables/
./variables/variables.data-00000-of-00001
 99  745M   99  744M    0     0  45.9M      0  0:00:16  0:00:16 --:--:-- 46.1M./variables/variables.index
./assets/
./saved_model.pb
100  745M  100  745M    0     0  45.9M      0  0:00:16  0:00:16 --:--:-- 47.5M


In [ ]:
import torch
!pip install transformers
!pip install --no-cache-dir transformers sentencepiece
#from transformers import BertForQuestionAnswering
#from transformers import BertTokenizer
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

torch_device = 'cuda'

#dmis-lab/biobert-base-cased-v1.1-squad
#abhijithneilabraham/longformer_covid_qa
#graviraja/covidbert_squad 

QA_TOKENIZER = AutoTokenizer.from_pretrained("dmis-lab/biobert-base-cased-v1.1-squad")
QA_MODEL = AutoModelForQuestionAnswering.from_pretrained("dmis-lab/biobert-base-cased-v1.1-squad")

#QA_MODEL = AutoModelForQuestionAnswering.from_pretrained("NeuML/bert-small-cord19qa")
#QA_TOKENIZER = AutoTokenizer.from_pretrained("NeuML/bert-small-cord19qa")

#QA_MODEL = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
#QA_TOKENIZER = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
QA_MODEL.to(torch_device)
QA_MODEL.eval()
#Auto Summarization with BART
if USE_SUMMARY:
    SUMMARY_TOKENIZER = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    SUMMARY_MODEL = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

    SUMMARY_MODEL.to(torch_device)
    SUMMARY_MODEL.eval()

In [ ]:
#%%capture
!pip install pyserini==0.8.1.0
from pyserini.search import pysearch

In [ ]:
query_processed = 'As we have mentioned above, COVID-19 is only fatal for a small percentage of people. In fact 80% face only mild symptoms with no need for hospitalization?'
query = 'As we have mentioned above, COVID-19 is only fatal for a small percentage of people. In fact 80% face only mild symptoms with no need for hospitalization?'

keywords = '2019-nCoV, SARS-CoV-2, COVID-19, symptoms, hospitalization'

In [ ]:
import json

searcher = pysearch.SimpleSearcher(luceneDir)
hits = searcher.search(query_processed + '. ' + keywords,k=50) # We can retrieve the documents according to concatanated string of query and keywords we can change this setup.
n_hits = len(hits)
## collect the relevant data in a hit dictionary
hit_dictionary = {}

In [ ]:
import json
#'As we have mentioned above, COVID-19 is only fatal for a small percentage of people. In fact 80% face only mild symptoms with no need for hospitalization?'

searcher = pysearch.SimpleSearcher(luceneDir)
hits = searcher.search(query_processed + '. ' + keywords,k=50)
n_hits = len(hits)
## collect the relevant data in a hit dictionary
hit_dictionary = {}
for i in range(0, n_hits):
  if 1==1:
    doc_json=dict()
    idx = str(hits[i].docid)
    hit_dictionary[idx] = doc_json
    hit_dictionary[idx]['abstract']=hits[i].lucene_document.get("raw")
    hit_dictionary[idx]['real_abstract']=hits[i].lucene_document.get("raw")
    hit_dictionary[idx]['title'] = str(hits[i].docid)

## scrub the abstracts in prep for BERT-SQuAD
for idx,v in hit_dictionary.items():
    abs_dirty = v['abstract']
    real_abs_dirty = v['real_abstract']
    #abs_dirty = v['paragraph']
    # looks like the abstract value can be an empty list
    v['abstract_paragraphs'] = []
    v['abstract_full'] = ''
    v['real_abstract_full'] = ''
    v['real_abstract_paragraphs']=[]

    if abs_dirty:
        # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
        # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
        # and a new entry that is full abstract text as both could be valuable for BERT derrived QA


        if isinstance(abs_dirty, list):
            for p in abs_dirty:
                v['abstract_paragraphs'].append(p['text'])
                v['abstract_full'] += p['text'] + ' \n\n'

        # looks like in some cases the abstract can be straight up text so we can actually leave that alone
        if isinstance(abs_dirty, str):
            v['abstract_paragraphs'].append(abs_dirty)
            v['abstract_full'] += abs_dirty + ' \n\n'
    if real_abs_dirty:
        # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
        # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
        # and a new entry that is full abstract text as both could be valuable for BERT derrived QA


        if isinstance(real_abs_dirty, list):
            for p in real_abs_dirty:
                v['real_abstract_paragraphs'].append(p['text'])
                v['real_abstract_full'] += p['text'] + ' \n\n'

        # looks like in some cases the abstract can be straight up text so we can actually leave that alone
        if isinstance(abs_dirty, str):
            v['real_abstract_paragraphs'].append(real_abs_dirty)
            v['real_abstract_full'] += real_abs_dirty + ' \n\n'
    v["indexed_para"]=v["abstract_full"][len(v["real_abstract_full"]):]

In [ ]:
def embed_useT(module):
    with tf.Graph().as_default():
        sentences = tf.compat.v1.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.compat.v1.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_useT('/root/kaggle/working/sentence_wise_email/module/module_useT')

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [ ]:
import numpy as np
#Reconstruct text for generating the answer text from the BERT result, do the necessary text preprocessing and construct the answer.
def reconstructText(tokens, start=0, stop=-1):
    tokens = tokens[start: stop]
    if '[SEP]' in tokens:
        sepind = tokens.index('[SEP]')
        tokens = tokens[sepind+1:]
    txt = ' '.join(tokens)
    txt = txt.replace(' ##', '')
    txt = txt.replace('##', '')
    txt = txt.strip()
    txt = " ".join(txt.split())
    txt = txt.replace(' .', '.')
    txt = txt.replace('( ', '(')
    txt = txt.replace(' )', ')')
    txt = txt.replace(' - ', '-')
    txt_list = txt.split(' , ')
    txt = ''
    nTxtL = len(txt_list)
    if nTxtL == 1:
        return txt_list[0]
    newList =[]
    for i,t in enumerate(txt_list):
        if i < nTxtL -1:
            if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                newList += [t,',']
            else:
                newList += [t, ', ']
        else:
            newList += [t]
    return ''.join(newList)


def makeBERTSQuADPrediction(document, question):
    ## we need to rewrite this function so that it chuncks the document into 250-300 word segments with
    ## 50 word overlaps on either end so that it can understand and check longer abstracts
    ## Get chuck of documents in order to overcome the BERT's max 512 token problem. First count the number of tokens. For example if it is
    ##   larger than 2048 token determine the split size and overlapping token count. Split the document into pieces to fed into BERT encoding. Overlapping
    ##      is necessary to maintain the coherence between the splits. If the token count is larger than some number 2560 token approximately this will fail.
    nWords = len(document.split())
    input_ids_all = QA_TOKENIZER.encode(question, document)
    tokens_all = QA_TOKENIZER.convert_ids_to_tokens(input_ids_all)
    overlapFac = 1.1
    if len(input_ids_all)*overlapFac > 2560:
        nSearchWords = int(np.ceil(nWords/6))
        fifth = int(np.ceil(nWords/5))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[fifth-int(nSearchWords*overlapFac/2):fifth+int(fifth*overlapFac/2)]),
                     ' '.join(docSplit[fifth*2-int(nSearchWords*overlapFac/2):fifth*2+int(fifth*overlapFac/2)]),
                     ' '.join(docSplit[fifth*3-int(nSearchWords*overlapFac/2):fifth*3+int(fifth*overlapFac/2)]),
                     ' '.join(docSplit[fifth*4-int(nSearchWords*overlapFac/2):fifth*4+int(fifth*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]

    elif len(input_ids_all)*overlapFac > 2048:
        nSearchWords = int(np.ceil(nWords/5))
        quarter = int(np.ceil(nWords/4))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[quarter-int(nSearchWords*overlapFac/2):quarter+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*2-int(nSearchWords*overlapFac/2):quarter*2+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[quarter*3-int(nSearchWords*overlapFac/2):quarter*3+int(quarter*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
        
    elif len(input_ids_all)*overlapFac > 1536:
        nSearchWords = int(np.ceil(nWords/4))
        third = int(np.ceil(nWords/3))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[third-int(nSearchWords*overlapFac/2):third+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[third*2-int(nSearchWords*overlapFac/2):third*2+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
        
    elif len(input_ids_all)*overlapFac > 1024:
        nSearchWords = int(np.ceil(nWords/3))
        middle = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                     ' '.join(docSplit[middle-int(nSearchWords*overlapFac/2):middle+int(nSearchWords*overlapFac/2)]),
                     ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    elif len(input_ids_all)*overlapFac > 512:
        nSearchWords = int(np.ceil(nWords/2))
        docSplit = document.split()
        docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
        input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
    else:
        input_ids = [input_ids_all]
    absTooLong = False    
    
    answers = []
    cons = []
    #print(input_ids)
    for iptIds in input_ids:
        tokens = QA_TOKENIZER.convert_ids_to_tokens(iptIds)
        #print(tokens)
        sep_index = iptIds.index(QA_TOKENIZER.sep_token_id)
        num_seg_a = sep_index + 1
        num_seg_b = len(iptIds) - num_seg_a
        segment_ids = [0]*num_seg_a + [1]*num_seg_b
        assert len(segment_ids) == len(iptIds)
        n_ids = len(segment_ids)
        #print(n_ids)
        if n_ids < 512:
            outputs = QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                                     token_type_ids=torch.tensor([segment_ids]).to(torch_device))
            #print(QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
             #                        token_type_ids=torch.tensor([segment_ids]).to(torch_device)))
        else:
            #this cuts off the text if its more than 512 words so it fits in model space
            #need run multiple inferences for longer text. add to the todo. I will try to expand this to longer sequences if it is necessary. 
            print('****** warning only considering first 512 tokens, document is '+str(nWords)+' words long.  There are '+str(n_ids)+ ' tokens')
            absTooLong = True
            outputs= QA_MODEL(torch.tensor([iptIds[:512]]).to(torch_device), 
                                     token_type_ids=torch.tensor([segment_ids[:512]]).to(torch_device))
        start_scores=outputs.start_logits
        end_scores=outputs.end_logits
        start_scores = start_scores[:,1:-1]
        end_scores = end_scores[:,1:-1]
        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        #print(answer_start, answer_end)
        answer = reconstructText(tokens, answer_start, answer_end+2)
    
        if answer.startswith('. ') or answer.startswith(', '):
            answer = answer[2:]
            
        c = start_scores[0,answer_start].item()+end_scores[0,answer_end].item()
        answers.append(answer)
        cons.append(c)
    
    maxC = max(cons)
    iMaxC = [i for i, j in enumerate(cons) if j == maxC][0]
    confidence = cons[iMaxC]
    answer = answers[iMaxC]
    
    sep_index = tokens_all.index('[SEP]')
    full_txt_tokens = tokens_all[sep_index+1:]
    
    abs_returned = reconstructText(full_txt_tokens)

    ans={}
    ans['answer'] = answer
    #print(answer)
    if answer.startswith('[CLS]') or answer_end.item() < sep_index or answer.endswith('[SEP]'):
        ans['confidence'] = -1000000
    else:
        #confidence = torch.max(start_scores) + torch.max(end_scores)
        #confidence = np.log(confidence.item())
        ans['confidence'] = confidence
    #ans['start'] = answer_start.item()
    #ans['end'] = answer_end.item()
    ans['abstract_bert'] = abs_returned
    ans['abs_too_long'] = absTooLong
    return ans

In [ ]:
from tqdm import tqdm
def searchAbstracts(hit_dictionary, question):
    abstractResults = {}
    for k,v in tqdm(hit_dictionary.items()):
        abstract = v['abstract_full']
        indexed_para=v['indexed_para']
        if abstract:
            ans = makeBERTSQuADPrediction(abstract, question)
            if ans['answer']:
                confidence = ans['confidence']
                abstractResults[confidence]={}
                abstractResults[confidence]['answer'] = ans['answer']
                #abstractResults[confidence]['start'] = ans['start']
                #abstractResults[confidence]['end'] = ans['end']
                abstractResults[confidence]['abstract_bert'] = ans['abstract_bert']
                abstractResults[confidence]['idx'] = k
                abstractResults[confidence]['abs_too_long'] = ans['abs_too_long']

                
    cList = list(abstractResults.keys())

    if cList:
        maxScore = max(cList)
        total = 0.0
        exp_scores = []
        for c in cList:
            s = np.exp(c-maxScore)
            exp_scores.append(s)
        total = sum(exp_scores)
        for i,c in enumerate(cList):
            abstractResults[exp_scores[i]/total] = abstractResults.pop(c)
    return abstractResults

In [ ]:
answers = searchAbstracts(hit_dictionary, query)

  0%|          | 0/50 [00:00<?, ?it/s]

****** warning only considering first 512 tokens, document is 3836 words long.  There are 1552 tokens
****** warning only considering first 512 tokens, document is 3836 words long.  There are 1730 tokens
****** warning only considering first 512 tokens, document is 3836 words long.  There are 1409 tokens
****** warning only considering first 512 tokens, document is 3836 words long.  There are 1689 tokens
****** warning only considering first 512 tokens, document is 3836 words long.  There are 1800 tokens


  2%|▏         | 1/50 [00:00<00:29,  1.66it/s]

****** warning only considering first 512 tokens, document is 3836 words long.  There are 1951 tokens
****** warning only considering first 512 tokens, document is 2444 words long.  There are 718 tokens
****** warning only considering first 512 tokens, document is 2444 words long.  There are 781 tokens
****** warning only considering first 512 tokens, document is 2444 words long.  There are 806 tokens


  4%|▍         | 2/50 [00:00<00:24,  1.95it/s]

...

****** warning only considering first 512 tokens, document is 6984 words long.  There are 2726 tokens
****** warning only considering first 512 tokens, document is 4758 words long.  There are 1572 tokens
****** warning only considering first 512 tokens, document is 4758 words long.  There are 1780 tokens


 98%|█████████▊| 49/50 [00:17<00:00,  3.33it/s]

****** warning only considering first 512 tokens, document is 4758 words long.  There are 1771 tokens
****** warning only considering first 512 tokens, document is 4758 words long.  There are 1490 tokens
****** warning only considering first 512 tokens, document is 4758 words long.  There are 1613 tokens
****** warning only considering first 512 tokens, document is 4758 words long.  There are 1495 tokens


100%|██████████| 50/50 [00:17<00:00,  2.81it/s]

****** warning only considering first 512 tokens, document is 4507 words long.  There are 1286 tokens
****** warning only considering first 512 tokens, document is 4507 words long.  There are 1331 tokens
****** warning only considering first 512 tokens, document is 4507 words long.  There are 1371 tokens
****** warning only considering first 512 tokens, document is 4507 words long.  There are 1404 tokens
****** warning only considering first 512 tokens, document is 4507 words long.  There are 1466 tokens
****** warning only considering first 512 tokens, document is 4507 words long.  There are 1450 tokens


In [ ]:
FIND_PDFS=False 
SEARCH_MEDRXIV=False

In [ ]:
workingPath = '/root/kaggle/working'
import pandas as pd
import re
if FIND_PDFS:
    from metapub import UrlReverse
    from metapub import FindIt
from IPython.core.display import display, HTML

#from summarizer import Summarizer
#summarizerModel = Summarizer()
def displayResults(hit_dictionary, answers, question):
    
    question_HTML = '<div style="font-family: Times New Roman; font-size: 28px; padding-bottom:28px"><b>Query</b>: '+question+'</div>'
    #all_HTML_txt = question_HTML
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    

    for c in confidence:
        if c>0 and c <= 1 and len(answers[c]['answer']) != 0:
            if 'idx' not in  answers[c]:
                continue
            rowData = []
            idx = answers[c]['idx']
            title = hit_dictionary[idx]['title']

            
            full_abs = answers[c]['abstract_bert']
            bert_ans = answers[c]['answer']
            
            
            #split_abs = full_abs.split(bert_ans)
            #x=(split_abs[0][:split_abs[0].rfind('. ')+1])
            #sentance_beginning = x[x.rfind('. ')+1:] + split_abs[0][split_abs[0].rfind('. ')+1:]
            #sentance_beginning = split_abs[0][split_abs[0].rfind('.')+1:]
            x=''
            y=''
            z=''
            t=''
            regex = r"(\.\s[^0-9])(?!.*(\.\s[^0-9]))"
            split_abs = full_abs.split(bert_ans)
            matches = re.finditer(regex, split_abs[0], re.MULTILINE)
            for matchNum1, match in enumerate(matches, start=1):
              y=(split_abs[0][:match.start()+1])
            matches = re.finditer(regex, y, re.MULTILINE)
            for matchNum2, match in enumerate(matches, start=1):
              x=(y[match.start()+1:])
            if x=='':
              x=split_abs[0]
            matches = re.finditer(regex, split_abs[0], re.MULTILINE)
            for matchNum3, match in enumerate(matches, start=1):
              z=(split_abs[0][match.start()+1:])  
            sentance_beginning = x+z
            regex2=r"(.*?(?<!\b\w)[.?!])\s+[a-zA-Z0-9]"
            if len(split_abs) == 1:
                sentance_end_pos = len(full_abs)
                sentance_end =''
            else:
                matches = re.finditer(regex2, split_abs[1], re.MULTILINE)
                for matchNum4, match in enumerate(matches, start=1):
                    if matchNum4==1:
                        t=(split_abs[1][:match.end()-2])
                sentance_end_pos = split_abs[1].find('. ')+1
                if sentance_end_pos == 0:
                    sentance_end = split_abs[1]
                else:
                    sentance_end = t
            #if len(split_abs) == 1:
            #    sentance_end_pos = len(full_abs)
            #    sentance_end =''
            #else:
            #    sentance_end_pos = split_abs[1].find('. ')+1
            #    if sentance_end_pos == 0:
            #      sentance_end = split_abs[1]
            #    else:
            #      sentance_end = split_abs[1][:sentance_end_pos]
                
            #sentance_full = sentance_beginning + bert_ans+ sentance_end
            answers[c]['full_answer'] = sentance_beginning+bert_ans+sentance_end
            answers[c]['partial_answer'] = bert_ans+sentance_end
            answers[c]['sentence_beginning'] = sentance_beginning
            answers[c]['sentence_end'] = sentance_end
            answers[c]['title'] = title
        else:
            answers.pop(c)
    
    
    ## now rerank based on semantic similarity of the answers to the question
    ## Universal sentence encoder
    cList = list(answers.keys())
    allAnswers = [answers[c]['full_answer'] for c in cList]
    
    messages = [question]+allAnswers
    
    encoding_matrix = embed_fn(messages)
    similarity_matrix = np.inner(encoding_matrix, encoding_matrix)
    rankings = similarity_matrix[1:,0]
    
    for i,c in enumerate(cList):
        answers[rankings[i]] = answers.pop(c)

    ## now form pandas dv
    confidence = list(answers.keys())
    confidence.sort(reverse=True)
    pandasData = []
    ranked_aswers = []
    full_abs_list=[]
    for c in confidence:
        rowData=[]
        title = answers[c]['title']
        idx = answers[c]['idx']
        rowData += [idx]            
        sentance_html = '<div>' +answers[c]['sentence_beginning'] + " <font color='red'>"+answers[c]['answer']+"</font> "+answers[c]['sentence_end']+'</div>'
        
        rowData += [sentance_html, c,title]
        pandasData.append(rowData)
        ranked_aswers.append(' '.join([answers[c]['full_answer']]))
        full_abs_list.append(' '.join([answers[c]['abstract_bert']]))
    
    if FIND_PDFS or SEARCH_MEDRXIV:
        pdata2 = []
        pm_ids = []
        for rowData in pandasData:
            rd = rowData
            idx = rowData[0]
    else:
        pdata2 = pandasData
        
    
    display(HTML(question_HTML))

    if USE_SUMMARY:
        ## try generating an exacutive summary with bart abstractive summarizer
        allAnswersTxt = ' '.join(ranked_aswers[:10]).replace('\n','')
    #    exec_sum = summarizerModel(allAnswersTxt, min_length=1, max_length=500)    
     #   execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>BERT Extractive Summary:</b>: '+exec_sum+'</div>'

        answers_input_ids = SUMMARY_TOKENIZER.batch_encode_plus([allAnswersTxt], return_tensors='pt', max_length=2048)['input_ids'].to(torch_device)
        #answers_input_ids = SUMMARY_TOKENIZER.prepare_seq2seq_batch([allAnswersTxt], return_tensors='pt', truncation=True, padding='longest')['input_ids'].to(torch_device)
        summary_ids = SUMMARY_MODEL.generate(answers_input_ids,
                                               num_beams=10,
                                               length_penalty=1.1,
                                               max_length=2048,
                                               min_length=64,
                                               no_repeat_ngram_size=0,
                                                do_sample=False )

        exec_sum = SUMMARY_TOKENIZER.decode(summary_ids.squeeze(), skip_special_tokens=True)
        execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; margin-bottom:1pt"><b>BART Abstractive Summary:</b>: '+exec_sum+'</div>'
        display(HTML(execSum_HTML))
        warning_HTML = '<div style="font-family: Times New Roman; font-size: 12px; padding-bottom:12px; color:#CCCC00; margin-top:1pt"> Warning this is an autogenerated summary based on semantic search of abstracts, always examine the sources before accepting this conclusion.  If the evidence only mentions topic in passing or the evidence is not clear, the summary will likely not clearly answer the question.</div>'
        display(HTML(warning_HTML))

#    display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>Body of Evidence:</b></div>'))
    
    if FIND_PDFS or SEARCH_MEDRXIV:
        df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])
    else:
        df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])
        
    display(HTML(df.to_html(render_links=True, escape=False)))
    return full_abs_list,ranked_aswers,pandasData

    


In [ ]:
full_abs_list,ranked_aswers,pm_ids=displayResults(hit_dictionary, answers, query)

In [ ]:
!pip install sentencepiece
import sentencepiece
from transformers import BartTokenizer, BartForConditionalGeneration
SUMMARY_TOKENIZER = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')
SUMMARY_MODEL = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')

#from transformers import PegasusTokenizer, PegasusForConditionalGeneration
#SUMMARY_TOKENIZER = PegasusTokenizer.from_pretrained('mayu0007/pegasus_large_covid')
#SUMMARY_MODEL = PegasusForConditionalGeneration.from_pretrained('mayu0007/pegasus_large_covid')

SUMMARY_MODEL.to(torch_device)
SUMMARY_MODEL.eval()
def Summarizer(string_all):    
    if 1==1:
        ## try generating an exacutive summary with bart abstractive summarizer
        allAnswersTxt = string_all.replace('\n','')

        answers_input_ids = SUMMARY_TOKENIZER.batch_encode_plus([allAnswersTxt], return_tensors='pt', max_length=1024)['input_ids'].to(torch_device)
        #answers_input_ids = SUMMARY_TOKENIZER.prepare_seq2seq_batch([allAnswersTxt], return_tensors='pt', truncation=True, padding='longest')['input_ids'].to(torch_device)
        summary_ids = SUMMARY_MODEL.generate(answers_input_ids,
                                               num_beams=4,
                                               length_penalty=0.8,
                                               repetition_penalty=2.0,
                                               min_length=5,
                                               no_repeat_ngram_size=0,
                                                do_sample=False )

        exec_sum = SUMMARY_TOKENIZER.decode(summary_ids.squeeze(), skip_special_tokens=True)
        return exec_sum

# Program to find most frequent  
# element in a list 
  
from collections import Counter 
  
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0]

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,AutoModelForSequenceClassification
#hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/albert-xxlarge-v2-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/bart-large-snli_mnli_fever_anli_R1_R2_R3-nli"
# hg_model_hub_name = "ynie/electra-large-discriminator-snli_mnli_fever_anli_R1_R2_R3-nli"
hg_model_hub_name = "ynie/xlnet-large-cased-snli_mnli_fever_anli_R1_R2_R3-nli"

tokenizer_nli = AutoTokenizer.from_pretrained(hg_model_hub_name, use_fast=False)
model_nli = AutoModelForSequenceClassification.from_pretrained(hg_model_hub_name)
model_nli.to(torch_device)
model_nli.eval()

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 1024)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=1024, out_features=4096, bias=True)
          (layer_2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,),

In [ ]:
def CheckEntailmentNeturalorContradict(ranked_aswers,string_all,pandasData):
  if 1==1:
    premise_list=[]
    hypothesis_list=[]
    entailment_prob=[]
    neutral_prob=[]
    contradict_prob=[]
    final_result_list=[]
    if len(ranked_aswers)>8:
      v_range=8
    else:
      v_range=len(ranked_aswers)
    for i in range(v_range):
      if pandasData[i][2]>=0.4:
        max_length = 512
        candidate_answer=Summarizer(ranked_aswers[i])
        #premise = "symptoms of covid-19 range from none (asymptomatic) to severe pneumonia and it can be fatal. fever and cough are the most common symptoms in patients with 2019-ncov infection. Several people have muscle soreness or fatigue as well as ards. diarrhea, hemoptysis, headache, sore throat, shock, and other symptoms only occur in a small number of patients"
        if len(candidate_answer)>len(ranked_aswers[i]):
          premise = ranked_aswers[i]
        else:
          premise=candidate_answer
        #premise = "covid-19 symptom is highly various in each patient, with fever, fatigue, shortness of breath, and cough as the main presenting symptoms. patient with covid-19 may shows severe symptom with severe pneumonia and ards, mild symptom resembling simple upper respiration tract infection, or even completely asymptomatic. approximately 80 % of cases is mild "
        hypothesis= string_all 
        tokenized_input_seq_pair = tokenizer_nli.encode_plus(premise, hypothesis,
                                                      max_length=max_length,
                                                      return_token_type_ids=True, truncation=True)

        input_ids = torch.Tensor(tokenized_input_seq_pair['input_ids']).long().unsqueeze(0).to(torch_device)
        # remember bart doesn't have 'token_type_ids', remove the line below if you are using bart.
        token_type_ids = torch.Tensor(tokenized_input_seq_pair['token_type_ids']).long().unsqueeze(0).to(torch_device)
        attention_mask = torch.Tensor(tokenized_input_seq_pair['attention_mask']).long().unsqueeze(0).to(torch_device)

        outputs = model_nli(input_ids,
                      attention_mask=attention_mask,
                      token_type_ids=token_type_ids,
                      labels=None)
        # Note:
        # "id2label": {
        #     "0": "entailment",
        #     "1": "neutral",
        #     "2": "contradiction"
        # },

        predicted_probability = torch.softmax(outputs[0], dim=1)[0].tolist()  # batch_size only one

        # print("Premise:", premise)
        premise_list.append(premise)
        # print("Hypothesis:", hypothesis)
        hypothesis_list.append(hypothesis)
        # print("input Ans:",ranked_aswers[i])
        # print("Entailment:", predicted_probability[0])
        entailment_prob.append(predicted_probability[0])
        # print("Neutral:", predicted_probability[1])
        neutral_prob.append(predicted_probability[1])
        # print("Contradiction:", predicted_probability[2])
        contradict_prob.append(predicted_probability[2])
    if len(premise_list)>0:
      avg_entailment=np.average(entailment_prob)
      avg_neutral=np.average(neutral_prob)
      avg_contradiction=np.average(contradict_prob) 
      zipped_list = zip(entailment_prob,neutral_prob,contradict_prob)
      for key,val in enumerate(hypothesis_list):
        if neutral_prob[key]>0.8:
          final_result='Neutral'
        elif entailment_prob[key]>contradict_prob[key]:
          final_result='True'
          final_result_list.append(final_result)
        else:
          final_result='False'
          final_result_list.append(final_result)
      if len(final_result_list)>0:
          verdict=most_frequent(final_result_list)
      else :
          verdict='Neutral'
    else:
      zipped_list = zip(0,1,0)
      verdict='Neutral'
  return zipped_list,verdict

# Test

In [ ]:
#!pip install pyserini==0.8.1.0
from pyserini.search import pysearch

import re
import sys
!pip install anglicize
from anglicize import anglicize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
!python -m pip install -U spacy
from spacy.tokenizer import Tokenizer
from nltk.tokenize import word_tokenize
from spacy.lang.en import English
import spacy
#!python -m spacy download en_core_web_sm
import spacy.cli # Colab fix
spacy.cli.download("en_core_web_sm") # Colab fix
import en_core_web_sm
nlp = en_core_web_sm.load()
nlp.add_pipe("merge_noun_chunks")
# nlp.add_pipe(nlp.create_pipe('merge_noun_chunks')) # Colab fix?
tokenizer = Tokenizer(nlp.vocab)
from timeit import default_timer as timer
import time
import json
import numpy as np
import statistics
import pandas as pd

DATA_TRAIN = "/content/train.jsonl" # Change this
DATA_DEV = "/content/dev.jsonl" # Change this

dataset_path = DATA_DEV
#dataset_path = DATA_TRAIN

with open(dataset_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
luceneDir = 'indexes/fever/lucene-index-fever-paragraph'
searcher = pysearch.SimpleSearcher(luceneDir)

def get_relevant_pages(claim, FULL_CLAIM, SPACY_ENTITIES, CASE_ENTITIES, ADD_SYNONYMS, ADD_SIMILAR, 
          LINKED_PAGES, LINKING_PAGES, CATCH_VERBS, TFIDF_TOP, FIND_ALL, N_RESULTS=70, SIMILAR_THRESHOLD=0.85, TFIDF_TOP_N=3):
    
    if LINKING_PAGES:
        import sqlalchemy as sqla

        db_fullpath = "D:/FEVER/feverous_wikiv1.db" # Change this
        db = sqla.create_engine("sqlite:///{}".format(db_fullpath))

        from sqlalchemy.orm import sessionmaker

        Session = sessionmaker(bind=db)
        session = Session()
    
    if not SPACY_ENTITIES and not CASE_ENTITIES and not LINKED_PAGES:
        FULL_CLAIM = True
    
    start_timer = timer()

    claim = nlp(claim.replace(" ", " ").replace("­", " ")) # Replaces the obnoxious space character with normal space

    keywords = set()
    entities = set()

    if SPACY_ENTITIES:
        spacy_entities = [entity.text for entity in claim.ents]
        entities.update(spacy_entities)

    if CASE_ENTITIES:
        case_entities = set()
        chunks = claim.noun_chunks
        for chunk in chunks:
            for token in tokenizer(chunk.text):
                if token.text[0].isupper():
                    case_entities.add(chunk.text)
                    break

        entities.update(case_entities)
        #print(case_entities)
        #print(entities)
        #sys.exit(0)

    keywords.update(entities)
    #print(keywords)

    if ADD_SYNONYMS:
        for token in claim:
            if token.is_stop:
                continue
            synonyms = wn.synsets(token.text)
            if synonyms and token:
                for synonym in synonyms:
                    if synonym.pos() == token.pos_[0].lower() and synonym.pos() == "n":
                        keywords.update([lemma.replace("_", " ") for lemma in synonym.lemma_names()])

    if ADD_SIMILAR:
        similar_check = [keyword for keyword in list(keywords) if keyword in vocab]

        if similar_check:
            similar_words = model.most_similar(positive=similar_check)
            for i in range(0, len(similar_words)):
                if i == 0 or similar_words[i][1] >= SIMILAR_THRESHOLD:
                    keywords.update([similar_words[i][0]])
    
    if CATCH_VERBS:
        for chunk in claim:
            if chunk.pos_ == "VERB" and chunk.dep_ != "case" and chunk.dep_ != "prep":
                keywords.update([chunk.lemma_])
                
    if TFIDF_TOP:
        tfidf_sorting = np.argsort(tfidf.transform([claim.text]).toarray()).flatten()[::-1]
        tfidf_len = len(tfidf_tokenize(claim.text))
        keywords.update(tfidf_features[tfidf_sorting][0:min([TFIDF_TOP_N, tfidf_len])])

    if not FULL_CLAIM:
        search_query = (", ".join(keywords) + '"' + '", "'.join(entities) + '"')
#            search_query = (", ".join(keywords) + ", ".join(entities))
    else:
        search_query = (claim.text + ' "' + '", "'.join(keywords) + '" "' + '", "'.join(entities) + '"')
#            search_query = (claim.text + " " + ", ".join(keywords) + ", ".join(entities))

    if FULL_CLAIM or keywords:
        try:
            lucene_hits = searcher.search(search_query.encode("utf-8"), k=N_RESULTS)
        except:
            lucene_hits = None
    else:
        lucene_hits = None

    hit_dictionary = {}

    if lucene_hits:

        for hit in lucene_hits:
            hit_dict = {"abstract": None, "real_abstract": None, "title": None}
            hit_dict['abstract']=hit.lucene_document.get("raw")
            hit_dict['real_abstract']=hit.lucene_document.get("raw")
            hit_dict['title'] = str(hit.docid)
            hit_dictionary[str(hit.docid)] = hit_dict
    
        linked_pages = set()
        if LINKED_PAGES:
            for hit in lucene_hits:
                links = re.findall(r"(?:\[\[)(.*?)(?:\|)", hit.raw)
                for link in links:
                    linked_pages.update([link.replace("_", " ").encode("latin-1", "ignore").decode("utf-8", "ignore")])
                    
        linking_pages = set()
        if LINKING_PAGES:
            linking_results = session.execute('select target, sources from inlinks where target IN ("{}")'.format('", "'.join([hit.docid.replace('"', '""') for hit in lucene_hits])))
            for row in linking_results:
                linking_pages.update(row["sources"].split(";"))
            """
            for hit in lucene_hits:
                linking_results = session.execute('select target, sources from inlinks where sources = "{}"'.format(hit.docid))
                for row in linking_results:
                    linking_pages.update(row["sources"].split(";"))"""

                    
        #print(lucene_hits[0].docid.encode("latin-1").decode("utf-8"))
        found_pages = [hit.docid for hit in lucene_hits]
        #found_pages = set([hit.docid.encode("latin-1", "ignore").decode("utf-8", "ignore") for page in lucene_hits])
        for i in range(0, len(found_pages)):

            try:
                found_pages[i] = found_pages[i].encode("latin-1", "ignore").decode("utf-8", "ignore")
                #print("Done:",found_pages[i])
            except:
                import regex
                print(found_pages[i], anglicize(found_pages[i]), regex.sub(r'[^\p{Latin}]', '', found_pages[i]).encode("latin-1").decode("utf-8"))
                
        found_pages = set(found_pages)

        hit_scores = [hit.score for hit in lucene_hits]
        hit_score_min = min(hit_scores)
        hit_score_25 = np.percentile(hit_scores, 25)
        hit_score_mean = np.mean(hit_scores)
        hit_score_median = statistics.median(hit_scores)
        hit_score_75 = np.percentile(hit_scores, 75)
        hit_score_max = max(hit_scores)
    
    else:
        
        is_found = False
        found_pages = set()
        entities = set()
        keywords = set()
        linked_pages = set()
        lucene_hits = []
        hit_scores = []
        hit_score_min = None
        hit_score_25 = None
        hit_score_mean = None
        hit_score_median = None
        hit_score_75 = None
        hit_score_max = None

    end_timer = timer()
    elapsed = int(end_timer - start_timer)
    elapsed_formatted = time.strftime("%H:%M:%S", time.gmtime(elapsed))
        
    result = [claim.text, elapsed_formatted, 
                found_pages, 
                keywords, 
                linked_pages, 
                linking_pages,
                len(lucene_hits), 
                hit_score_min, hit_score_25, hit_score_mean, hit_score_median, 
                hit_score_75, hit_score_max, hit_scores]

    #print(result)
    
    result = pd.DataFrame([result], columns=["CLAIM", "ELAPSED", 
                                             "PAGES_FOUND",
                                             "KEYWORDS", 
                                             "LINKED_PAGES",
                                             "LINKING_PAGES",
                                             "N_LUCENE_HITS", 
                                             "HIT_SCORE_MIN", "HIT_SCORE_25", "HIT_SCORE_MEAN", "HIT_SCORE_MEDIAN", 
                                             "HIT_SCORE_75", "HIT_SCORE_MAX", "HIT_SCORES"])
    
    return result, hit_dictionary

In [ ]:
hit_stats, hit_dictionary = get_relevant_pages(dataset[0]["claim"], FULL_CLAIM=True, SPACY_ENTITIES=True, CASE_ENTITIES=True, ADD_SYNONYMS=False, ADD_SIMILAR=False, 
            LINKED_PAGES=False, LINKING_PAGES=False, CATCH_VERBS=False, TFIDF_TOP=False, FIND_ALL=True, N_RESULTS=70)

print([key for key in hit_dictionary])
hit_stats

['Ailanthus', 'Myrica', 'Spiraea', 'Izatha dulcior', 'Metacrias huttoni', 'Kon Ka Kinh National Park', 'Acacia sensu lato', 'Metacrias erichrysa', 'Metacrias strategica', 'William Trowbridge Merrifield Forbes', 'Alexander Julius Reichert', 'Julius Lederer (entomologist)', 'Museum of Natural History, University of Wrocław', 'Roscoea', 'Succession of the 14th Dalai Lama', 'Palace lantern', 'Alder', 'Lathys', 'Migration in China', 'Apoptosis', 'Agabus aequalis', 'Agilisaurus', 'Plumeria rubra', 'Xerochrysum bracteatum', 'Kuomintang', 'Jack Belden', 'Bambusiomyces', 'Elephantoceras', 'Amway', 'De-Sinicization', 'Antoninus Pius', 'Gim Jeong-hui', 'Battle of Pengcheng', 'Asiloidea', 'Khmer Rouge', 'Mary Jean Eisenhower', 'Yoshihiko Noda', 'Language and overseas Chinese communities', 'Internment of Chinese-Indians', 'Liugong Island', 'Ang (surname)', 'Sunken Plum', 'Liuli Gongfang', 'List of disputed territories of India', 'Li Qiang (activist)', 'Singaporemma', 'Qiongthela', 'Anna Chennault',

,CLAIM,ELAPSED,PAGES_FOUND,KEYWORDS,LINKED_PAGES,LINKING_PAGES,N_LUCENE_HITS,HIT_SCORE_MIN,HIT_SCORE_25,HIT_SCORE_MEAN,HIT_SCORE_MEDIAN,HIT_SCORE_75,HIT_SCORE_MAX,HIT_SCORES
0,Tridrepana hainana is a moth found in China th...,00:00:00,"{Agabus aequalis, Sunken Plum, Myrica, Kunio K...","{Tridrepana hainana, the Drepanidae family, Ch...",{},{},70,11.7252,12.029525,14.494609,12.4629,13.758575,33.459202,"[33.45920181274414, 30.453800201416016, 25.158..."


In [ ]:
def get_answers(query, hit_dictionary):
    for idx,v in hit_dictionary.items():
        abs_dirty = v['abstract']
        real_abs_dirty = v['real_abstract']
        #abs_dirty = v['paragraph']
        # looks like the abstract value can be an empty list
        v['abstract_paragraphs'] = []
        v['abstract_full'] = ''
        v['real_abstract_full'] = ''
        v['real_abstract_paragraphs']=[]

        if abs_dirty:
            # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
            # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
            # and a new entry that is full abstract text as both could be valuable for BERT derrived QA

            if isinstance(abs_dirty, list):
                for p in abs_dirty:
                    v['abstract_paragraphs'].append(p['text'])
                    v['abstract_full'] += p['text'] + ' \n\n'

            # looks like in some cases the abstract can be straight up text so we can actually leave that alone
            if isinstance(abs_dirty, str):
                v['abstract_paragraphs'].append(abs_dirty)
                v['abstract_full'] += abs_dirty + ' \n\n'
        if real_abs_dirty:
            # looks like if it is a list, then the only entry is a dictionary wher text is in 'text' key
            # looks like it is broken up by paragraph if it is in that form.  lets make lists for every paragraph
            # and a new entry that is full abstract text as both could be valuable for BERT derrived QA


            if isinstance(real_abs_dirty, list):
                for p in real_abs_dirty:
                    v['real_abstract_paragraphs'].append(p['text'])
                    v['real_abstract_full'] += p['text'] + ' \n\n'

            # looks like in some cases the abstract can be straight up text so we can actually leave that alone
            if isinstance(abs_dirty, str):
                v['real_abstract_paragraphs'].append(real_abs_dirty)
                v['real_abstract_full'] += real_abs_dirty + ' \n\n'
        v["indexed_para"]=v["abstract_full"][len(v["real_abstract_full"]):]

    def embed_useT(module):
        with tf.Graph().as_default():
            sentences = tf.compat.v1.placeholder(tf.string)
            embed = hub.Module(module)
            embeddings = embed(sentences)
            session = tf.compat.v1.train.MonitoredSession()
        return lambda x: session.run(embeddings, {sentences: x})
    embed_fn = embed_useT('/root/kaggle/working/sentence_wise_email/module/module_useT')

    import numpy as np
    #Reconstruct text for generating the answer text from the BERT result, do the necessary text preprocessing and construct the answer.
    def reconstructText(tokens, start=0, stop=-1):
        tokens = tokens[start: stop]
        if '[SEP]' in tokens:
            sepind = tokens.index('[SEP]')
            tokens = tokens[sepind+1:]
        txt = ' '.join(tokens)
        txt = txt.replace(' ##', '')
        txt = txt.replace('##', '')
        txt = txt.strip()
        txt = " ".join(txt.split())
        txt = txt.replace(' .', '.')
        txt = txt.replace('( ', '(')
        txt = txt.replace(' )', ')')
        txt = txt.replace(' - ', '-')
        txt_list = txt.split(' , ')
        txt = ''
        nTxtL = len(txt_list)
        if nTxtL == 1:
            return txt_list[0]
        newList =[]
        for i,t in enumerate(txt_list):
            if i < nTxtL -1:
                if t[-1].isdigit() and txt_list[i+1][0].isdigit():
                    newList += [t,',']
                else:
                    newList += [t, ', ']
            else:
                newList += [t]
        return ''.join(newList)


    def makeBERTSQuADPrediction(document, question):
        ## we need to rewrite this function so that it chuncks the document into 250-300 word segments with
        ## 50 word overlaps on either end so that it can understand and check longer abstracts
        ## Get chuck of documents in order to overcome the BERT's max 512 token problem. First count the number of tokens. For example if it is
        ##   larger than 2048 token determine the split size and overlapping token count. Split the document into pieces to fed into BERT encoding. Overlapping
        ##      is necessary to maintain the coherence between the splits. If the token count is larger than some number 2560 token approximately this will fail.
        nWords = len(document.split())
        input_ids_all = QA_TOKENIZER.encode(question, document)
        tokens_all = QA_TOKENIZER.convert_ids_to_tokens(input_ids_all)
        overlapFac = 1.1
        if len(input_ids_all)*overlapFac > 2560:
            nSearchWords = int(np.ceil(nWords/6))
            fifth = int(np.ceil(nWords/5))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[fifth-int(nSearchWords*overlapFac/2):fifth+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[fifth*2-int(nSearchWords*overlapFac/2):fifth*2+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[fifth*3-int(nSearchWords*overlapFac/2):fifth*3+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[fifth*4-int(nSearchWords*overlapFac/2):fifth*4+int(fifth*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]

        elif len(input_ids_all)*overlapFac > 2048:
            nSearchWords = int(np.ceil(nWords/5))
            quarter = int(np.ceil(nWords/4))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[quarter-int(nSearchWords*overlapFac/2):quarter+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[quarter*2-int(nSearchWords*overlapFac/2):quarter*2+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[quarter*3-int(nSearchWords*overlapFac/2):quarter*3+int(quarter*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
            
        elif len(input_ids_all)*overlapFac > 1536:
            nSearchWords = int(np.ceil(nWords/4))
            third = int(np.ceil(nWords/3))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[third-int(nSearchWords*overlapFac/2):third+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[third*2-int(nSearchWords*overlapFac/2):third*2+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]        
            
        elif len(input_ids_all)*overlapFac > 1024:
            nSearchWords = int(np.ceil(nWords/3))
            middle = int(np.ceil(nWords/2))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), 
                        ' '.join(docSplit[middle-int(nSearchWords*overlapFac/2):middle+int(nSearchWords*overlapFac/2)]),
                        ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        elif len(input_ids_all)*overlapFac > 512:
            nSearchWords = int(np.ceil(nWords/2))
            docSplit = document.split()
            docPieces = [' '.join(docSplit[:int(nSearchWords*overlapFac)]), ' '.join(docSplit[-int(nSearchWords*overlapFac):])]
            input_ids = [QA_TOKENIZER.encode(question, dp) for dp in docPieces]
        else:
            input_ids = [input_ids_all]
        absTooLong = False    
        
        answers = []
        cons = []
        #print(input_ids)
        for iptIds in input_ids:
            tokens = QA_TOKENIZER.convert_ids_to_tokens(iptIds)
            #print(tokens)
            sep_index = iptIds.index(QA_TOKENIZER.sep_token_id)
            num_seg_a = sep_index + 1
            num_seg_b = len(iptIds) - num_seg_a
            segment_ids = [0]*num_seg_a + [1]*num_seg_b
            assert len(segment_ids) == len(iptIds)
            n_ids = len(segment_ids)
            #print(n_ids)
            if n_ids < 512:
                outputs = QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                                        token_type_ids=torch.tensor([segment_ids]).to(torch_device))
                #print(QA_MODEL(torch.tensor([iptIds]).to(torch_device), 
                #                        token_type_ids=torch.tensor([segment_ids]).to(torch_device)))
            else:
                #this cuts off the text if its more than 512 words so it fits in model space
                #need run multiple inferences for longer text. add to the todo. I will try to expand this to longer sequences if it is necessary. 
                print('****** warning only considering first 512 tokens, document is '+str(nWords)+' words long.  There are '+str(n_ids)+ ' tokens')
                absTooLong = True
                outputs= QA_MODEL(torch.tensor([iptIds[:512]]).to(torch_device), 
                                        token_type_ids=torch.tensor([segment_ids[:512]]).to(torch_device))
            start_scores=outputs.start_logits
            end_scores=outputs.end_logits
            start_scores = start_scores[:,1:-1]
            end_scores = end_scores[:,1:-1]
            answer_start = torch.argmax(start_scores)
            answer_end = torch.argmax(end_scores)
            #print(answer_start, answer_end)
            answer = reconstructText(tokens, answer_start, answer_end+2)
        
            if answer.startswith('. ') or answer.startswith(', '):
                answer = answer[2:]
                
            c = start_scores[0,answer_start].item()+end_scores[0,answer_end].item()
            answers.append(answer)
            cons.append(c)
        
        maxC = max(cons)
        iMaxC = [i for i, j in enumerate(cons) if j == maxC][0]
        confidence = cons[iMaxC]
        answer = answers[iMaxC]
        
        sep_index = tokens_all.index('[SEP]')
        full_txt_tokens = tokens_all[sep_index+1:]
        
        abs_returned = reconstructText(full_txt_tokens)

        ans={}
        ans['answer'] = answer
        #print(answer)
        if answer.startswith('[CLS]') or answer_end.item() < sep_index or answer.endswith('[SEP]'):
            ans['confidence'] = -1000000
        else:
            #confidence = torch.max(start_scores) + torch.max(end_scores)
            #confidence = np.log(confidence.item())
            ans['confidence'] = confidence
        #ans['start'] = answer_start.item()
        #ans['end'] = answer_end.item()
        ans['abstract_bert'] = abs_returned
        ans['abs_too_long'] = absTooLong
        return ans

    from tqdm import tqdm
    def searchAbstracts(hit_dictionary, question):
        abstractResults = {}
        for k,v in tqdm(hit_dictionary.items()):
            abstract = v['abstract_full']
            indexed_para=v['indexed_para']
            if abstract:
                ans = makeBERTSQuADPrediction(abstract, question)
                if ans['answer']:
                    confidence = ans['confidence']
                    abstractResults[confidence]={}
                    abstractResults[confidence]['answer'] = ans['answer']
                    #abstractResults[confidence]['start'] = ans['start']
                    #abstractResults[confidence]['end'] = ans['end']
                    abstractResults[confidence]['abstract_bert'] = ans['abstract_bert']
                    abstractResults[confidence]['idx'] = k
                    abstractResults[confidence]['abs_too_long'] = ans['abs_too_long']

                    
        cList = list(abstractResults.keys())

        if cList:
            maxScore = max(cList)
            total = 0.0
            exp_scores = []
            for c in cList:
                s = np.exp(c-maxScore)
                exp_scores.append(s)
            total = sum(exp_scores)
            for i,c in enumerate(cList):
                abstractResults[exp_scores[i]/total] = abstractResults.pop(c)
        return abstractResults

    answers = searchAbstracts(hit_dictionary, query)
    
    FIND_PDFS=False 
    SEARCH_MEDRXIV=False

    workingPath = '/root/kaggle/working'
    import pandas as pd
    import re
    if FIND_PDFS:
        from metapub import UrlReverse
        from metapub import FindIt
    from IPython.core.display import display, HTML

    #from summarizer import Summarizer
    #summarizerModel = Summarizer()
    def displayResults(hit_dictionary, answers, question):
        
        question_HTML = '<div style="font-family: Times New Roman; font-size: 28px; padding-bottom:28px"><b>Query</b>: '+question+'</div>'
        #all_HTML_txt = question_HTML
        confidence = list(answers.keys())
        confidence.sort(reverse=True)
        
        confidence = list(answers.keys())
        confidence.sort(reverse=True)
        

        for c in confidence:
            if c>0 and c <= 1 and len(answers[c]['answer']) != 0:
                if 'idx' not in  answers[c]:
                    continue
                rowData = []
                idx = answers[c]['idx']
                title = hit_dictionary[idx]['title']

                
                full_abs = answers[c]['abstract_bert']
                bert_ans = answers[c]['answer']
                
                
                #split_abs = full_abs.split(bert_ans)
                #x=(split_abs[0][:split_abs[0].rfind('. ')+1])
                #sentance_beginning = x[x.rfind('. ')+1:] + split_abs[0][split_abs[0].rfind('. ')+1:]
                #sentance_beginning = split_abs[0][split_abs[0].rfind('.')+1:]
                x=''
                y=''
                z=''
                t=''

                # print(bert_ans)
                split_abs = full_abs.split(bert_ans)
                sentance_beginning = split_abs[0].split(" ~ ~ ")[-1]
                sentance_end = split_abs[1].split(" ~ ~ ")[0]
                    
                sentance_full = sentance_beginning + bert_ans+ sentance_end
                print(sentance_full)

                answer_ids = re.findall(r"((sentence|table|cell|section|list|item)( _ [0-9]+)+ )", sentance_full)
                answer_ids_formatted = []

                for ids in answer_ids:
                    if isinstance(ids, str):
                        candidate = ids.replace(" ", "")
                        if "_" in candidate and candidate[0] != "_" and candidate not in ["sentence", "table", "cell", "section", "list", "item"]:  # very hacky
                            answer_ids_formatted.append(candidate)
                    else:
                        for id in ids:
                            candidate = id.replace(" ", "")
                            if "_" in candidate and candidate[0] != "_" and candidate not in ["sentence", "table", "cell", "section", "list", "item"]:  # very hacky
                                answer_ids_formatted.append(candidate)
                print(answer_ids_formatted)

                answers[c]['full_answer'] = sentance_beginning+bert_ans+sentance_end
                answers[c]['partial_answer'] = bert_ans+sentance_end
                answers[c]['sentence_beginning'] = sentance_beginning
                answers[c]['sentence_end'] = sentance_end
                answers[c]['title'] = title
            else:
                answers.pop(c)
        
        
        ## now rerank based on semantic similarity of the answers to the question
        ## Universal sentence encoder
        cList = list(answers.keys())
        allAnswers = [answers[c]['full_answer'] for c in cList]
        
        messages = [question]+allAnswers
        
        encoding_matrix = embed_fn(messages)
        similarity_matrix = np.inner(encoding_matrix, encoding_matrix)
        rankings = similarity_matrix[1:,0]
        
        for i,c in enumerate(cList):
            answers[rankings[i]] = answers.pop(c)

        ## now form pandas dv
        confidence = list(answers.keys())
        confidence.sort(reverse=True)
        pandasData = []
        ranked_aswers = []
        full_abs_list=[]
        for c in confidence:
            rowData=[]
            title = answers[c]['title']
            idx = answers[c]['idx']
            rowData += [idx]            
            sentance_html = '<div>' +answers[c]['sentence_beginning'] + " <font color='red'>"+answers[c]['answer']+"</font> "+answers[c]['sentence_end']+'</div>'
            answer_key = answers[c]['answer'].split("\t")[-1].strip() if "\t" in answers[c]['answer'] else answers[c]['answer'].strip()

            # rowData += [sentance_html, answer_key, c,title]
            rowData += [sentance_html, c,title]
            pandasData.append(rowData)
            ranked_aswers.append(' '.join([answers[c]['full_answer']]))
            full_abs_list.append(' '.join([answers[c]['abstract_bert']]))
        
        if FIND_PDFS or SEARCH_MEDRXIV:
            pdata2 = []
            pm_ids = []
            for rowData in pandasData:
                rd = rowData
                idx = rowData[0]
        else:
            pdata2 = pandasData
            
        
        display(HTML(question_HTML))

        if USE_SUMMARY:
            ## try generating an exacutive summary with bart abstractive summarizer
            allAnswersTxt = ' '.join(ranked_aswers[:10]).replace('\n','')
        #    exec_sum = summarizerModel(allAnswersTxt, min_length=1, max_length=500)    
        #   execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>BERT Extractive Summary:</b>: '+exec_sum+'</div>'

            answers_input_ids = SUMMARY_TOKENIZER.batch_encode_plus([allAnswersTxt], return_tensors='pt', max_length=2048)['input_ids'].to(torch_device)
            #answers_input_ids = SUMMARY_TOKENIZER.prepare_seq2seq_batch([allAnswersTxt], return_tensors='pt', truncation=True, padding='longest')['input_ids'].to(torch_device)
            summary_ids = SUMMARY_MODEL.generate(answers_input_ids,
                                                num_beams=10,
                                                length_penalty=1.1,
                                                max_length=2048,
                                                min_length=64,
                                                no_repeat_ngram_size=0,
                                                    do_sample=False )

            exec_sum = SUMMARY_TOKENIZER.decode(summary_ids.squeeze(), skip_special_tokens=True)
            execSum_HTML = '<div style="font-family: Times New Roman; font-size: 18px; margin-bottom:1pt"><b>BART Abstractive Summary:</b>: '+exec_sum+'</div>'
            display(HTML(execSum_HTML))
            warning_HTML = '<div style="font-family: Times New Roman; font-size: 12px; padding-bottom:12px; color:#CCCC00; margin-top:1pt"> Warning this is an autogenerated summary based on semantic search of abstracts, always examine the sources before accepting this conclusion.  If the evidence only mentions topic in passing or the evidence is not clear, the summary will likely not clearly answer the question.</div>'
            display(HTML(warning_HTML))

    #    display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:18px"><b>Body of Evidence:</b></div>'))
        
        if FIND_PDFS or SEARCH_MEDRXIV:
            # df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Answer Key', 'Confidence', 'Title/Link'])
            df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])
        else:
            # df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Answer Key', 'Confidence', 'Title/Link'])
            df = pd.DataFrame(pdata2, columns = ['Lucene ID', 'BERT-SQuAD Answer with Highlights', 'Confidence', 'Title/Link'])
            
        display(HTML(df.to_html(render_links=True, escape=False)))
        return full_abs_list,ranked_aswers,pandasData

    return displayResults(hit_dictionary, answers, query)




In [ ]:
for data in dataset[:1]:
    claim = data["claim"]
    hit_stats, hit_dictionary = get_relevant_pages(claim, FULL_CLAIM=True, SPACY_ENTITIES=True, CASE_ENTITIES=True, ADD_SYNONYMS=False, ADD_SIMILAR=False, 
                            LINKED_PAGES=False, LINKING_PAGES=False, CATCH_VERBS=False, TFIDF_TOP=False, FIND_ALL=True, N_RESULTS=70)
    
    full_abs_list, ranked_aswers, pandasData = get_answers(claim, hit_dictionary)

    # counter = 0
    # for data in pandasData:
    #     if data[2] > 0.4:
    #         counter += 1
    # print(counter)

    evidence_probs, verdict = CheckEntailmentNeturalorContradict(ranked_aswers,claim,pandasData)

    # print(len(list(evidence_probs)))
    # print(verdict)

    if verdict == "False":
        predicted_label = "REFUTES"
    elif verdict == "True":
        predicted_label = "SUPPORTS"
    else:
        predicted_label = "NOT ENOUGH INFO"
    
    output = {"id": "", "predicted_label": predicted_label, "predicted_evidence": [[data[3], data[1], ""] for data in pandasData[:min([len(pandasData), 5])]]}

    print(output)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


  0%|          | 0/70 [00:00<?, ?it/s]

****** warning only considering first 512 tokens, document is 379 words long.  There are 514 tokens


  6%|▌         | 4/70 [00:00<00:06,  9.92it/s]

****** warning only considering first 512 tokens, document is 537 words long.  There are 544 tokens
****** warning only considering first 512 tokens, document is 537 words long.  There are 515 tokens
****** warning only considering first 512 tokens, document is 542 words long.  There are 516 tokens


...

****** warning only considering first 512 tokens, document is 7887 words long.  There are 3065 tokens
****** warning only considering first 512 tokens, document is 1229 words long.  There are 564 tokens
****** warning only considering first 512 tokens, document is 1229 words long.  There are 532 tokens
****** warning only considering first 512 tokens, document is 1229 words long.  There are 558 tokens


100%|██████████| 70/70 [00:12<00:00,  5.43it/s]


sentence _ 5 a traumatized young girl is found alive in the basement, later identified as cherry, who was abducted 12 years ago. ~ ~ sentence _ 6 her family, who has since resided in hong kong, feels agitated upon the shocking news. ~ ~ sentence _ 7 in the wake of the trauma, cherry appears to lose all recollection of her family and childhood. ~ ~ sentence _ 8 to cope with the aftermath and get her life back on track, her family takes her back to hong kong to start anew. ~ ~ sentence _ 9 while cherry is making progress in recovery, strange events occur that threaten to harm those close to her. ~ ~ sentence _ 10 complicated by the appearance of a mysterious stranger, police investigator starts to look into cherry ' s past for lead. ~ ~ sentence _ 11 but what he is about to discover, no one is prepared for the fatal outcome that follows... ~ ~ section _ 1 cast ~ ~ section _ 2 reception ~ ~ sentence _ 12 the film has grossed cna¥33. 3 million in china. ~ ~
['sentence_5', 'sentence_6', 'se

,Lucene ID,BERT-SQuAD Answer with Highlights,Confidence,Title/Link
0,Agabus aequalis,sentence _ 0 agabus aequalis is a species of predatory diving [ [ beetle | beetle ] ] belonging to the family [ [ dytiscidae | dytiscidae ] ].,0.796799,Agabus aequalis
1,Metacrias erichrysa,sentence _ 2 this species can be found in the lower half of the north island and western alpine areas of the south,0.691027,Metacrias erichrysa
2,Bambusiomyces,"sentence _ 0 bambusiomyces is a fungal [ [ genus | genus ] ] in the family [ [ ustilaginaceae | ustilaginaceae ] ]. ~ ~ sentence _ 1 it was circumscribed in 2011 to contain the [ [ smut _ fungus | smut fungus ] ] formerly known as ustilago shiraina, originally described by german mycologist [ [ paul _ christoph _ hennings | paul christoph hennings ] ] from japan in 1900. ~ ~ sentence _ 2 the fungus was originally found growing on wild japanese bamboo, bambusa veitchii, but has since been isolated from several plants belonging to tribe [ [ bambuseae | bambuseae ] ] in the grass family poaceae. ~ ~ sentence _ 3 in addition to japan, it is also found in china and other locales in southeast asia , and has been [ [ introduced _ species | introduced ] ] to the united states. ~ ~",0.685733,Bambusiomyces
3,Schima wallichii,"sentence _ 2 it is found in northern indian, [ [ bangladesh | bangladesh ] ], [ [ nepal | nepal ] ], [ [ bhutan | bhutan ] ], southwestern china ([ [ tibet | tibet ] ], [ [ yunnan | yunnan ] ], [ [ guizhou | guizhou ] ], and guangxi ), [ [ myanmar | myanmar ] ], [ [ thailand | thailand ] ], [ [ laos | laos ] ], and [ [ vietnam | vietnam ] ].",0.677549,Schima wallichii
4,Shouchangoceratinae,"sentence _ 0 shouchangoceratinae is one of three [ [ ammonoid | ammonoid ] ] [ [ subfamilies | subfamilies ] ] of the [ [ family _ (biology) | family ] ] [ [ pseudohaloritidae | pseudohaloritidae ] ], which in turn is one of two families in the [ [ goniatitida | goniatitid superfamily ] ] [ [ pseudohaloritoidea | pseudohaloritoidea ] ]. ~ ~ sentence _ 1 the shouchangoceratinid ammonoids were found in marine environments throughout the world during the [ [ permian | permian ] ], particularly in [ [ china | china ] ]. ~ ~",0.575411,Shouchangoceratinae
5,Elephantoceras,sentence _ 0 elephantoceras is genus of a [ [ middle _ permian | middle permian ] ] [ [ ammonoidea | ammonite ] ] belonging to the [ [ goniatitida | goniatitid ] ] family [ [ pseudohaloritidae | pseudohaloritidae ] ]. ~ ~ sentence _ 1 fossils belonging to this genera were found in china. ~ ~,0.555315,Elephantoceras
6,Singaporemma,"item _ 0 _ 2 singaporemma bifurcatum lin & li, 2010 – china ~ ~ item _ 0 _ 3 singaporemma halongense lehtinen , 1981 – vietnam",0.221622,Singaporemma
7,Cherry Returns,"sentence _ 5 a traumatized young girl is found alive in the basement, later identified as cherry, who was abducted 12 years ago. ~ ~ sentence _ 6 her family, who has since resided in hong kong, feels agitated upon the shocking news. ~ ~ sentence _ 7 in the wake of the trauma, cherry appears to lose all recollection of her family and childhood. ~ ~ sentence _ 8 to cope with the aftermath and get her life back on track, her family takes her back to hong kong to start anew. ~ ~ sentence _ 9 while cherry is making progress in recovery, strange events occur that threaten to harm those close to her. ~ ~ sentence _ 10 complicated by the appearance of a mysterious stranger, police investigator starts to look into cherry ' s past for lead. ~ ~ sentence _ 11 but what he is about to discover, no one is prepared for the fatal outcome that follows... ~ ~ section _ 1 cast ~ ~ section _ 2 reception ~ ~ sentence _ 12 the film has grossed cna¥33. 3 million in china. ~ ~",0.180775,Cherry Returns


{'id': '', 'predicted_label': 'REFUTES', 'predicted_evidence': [['Agabus aequalis', "<div>sentence _  <font color='red'>0 agabus aequalis</font>  is a species of predatory diving [ [ beetle | beetle ] ] belonging to the family [ [ dytiscidae | dytiscidae ] ].</div>", ''], ['Metacrias erichrysa', "<div>sentence _ 2 this species can be found in the lower half of the north  <font color='red'>island</font>  and western alpine areas of the south </div>", ''], ['Bambusiomyces', "<div> <font color='red'>sentence _ 0 bambusiomyces is a fungal [ [ genus | genus ] ] in the family [ [ ustilaginaceae | ustilaginaceae ] ]. ~ ~ sentence _ 1 it was circumscribed in 2011 to contain the [ [ smut _ fungus | smut fungus ] ] formerly known as ustilago shiraina, originally described by german mycologist [ [ paul _ christoph _ hennings | paul christoph hennings ] ] from japan in 1900. ~ ~ sentence _ 2 the fungus was originally found growing on wild japanese bamboo, bambusa veitchii, but has since been isola